<a href="https://colab.research.google.com/github/JVR27XD/MobileSAM/blob/main/NANOSAM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /content/nanosam

In [2]:
# Clonar el repositorio
!git clone https://github.com/NVIDIA-AI-IOT/nanosam.git
%cd /content/nanosam


Cloning into 'nanosam'...
remote: Enumerating objects: 1309, done.
remote: Total 1309 (delta 0), reused 0 (delta 0), pack-reused 1309 (from 1)
Receiving objects: 100% (1309/1309), 97.56 MiB | 22.63 MiB/s, done.
Resolving deltas: 100% (749/749), done.
/content/nanosam


In [3]:
# ⚠️ Forzar numpy compatible
!pip install numpy==1.26.4 --force-reinstall

# Instalar ONNX + ONNXRuntime GPU (CUDA 12 para T4)
!pip install onnx==1.15.0 onnxruntime-gpu==1.17.1 --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/

# Instalar TensorRT 8.6 + PyCUDA
!pip install nvidia-pyindex nvidia-tensorrt==8.6.1.6 pycuda --no-cache-dir


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 100.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


Looking in indexes: https://pypi.org/simple, https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.9/192.9 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 850.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 181.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 105.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement nvidia-tensorrt==8.6.1.6 (from versions: 0.0.1.dev4, 0.0.1.dev5, 99.0.0)
ERROR: No matching distribution found for nvidia-tensorrt==8.6.1.6


In [4]:
# Montar Drive (para guardar el .engine)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Añadir carpeta nanosam al PYTHONPATH
import sys
sys.path.append("/content/nanosam")


In [6]:
%cd /content/nanosam
!pip install -e .


/content/nanosam
Obtaining file:///content/nanosam
  Preparing metadata (setup.py) ... done
  Running setup.py develop for nanosam


In [7]:
# Exportar modelo ONNX
!python3 nanosam/tools/export_sam_mask_decoder_onnx.py \
    --model-type vit_t \
    --checkpoint "/content/drive/MyDrive/Colab Notebooks/TFG/NanoSam/mobile_sam.pt" \
    --output "/content/drive/MyDrive/Colab Notebooks/TFG/NanoSam/mobile_sam_mask_decoder.onnx"

/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/content/nanosam/nanosam/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with nanosam.mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/content/nanosam/nanosam/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_11m_224 in registry with 

In [8]:
!pip install -U polygraphy --extra-index-url https://pypi.ngc.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 7.1 MB/s eta 0:00:00


In [9]:
!tar -xvzf "/content/drive/MyDrive/Colab Notebooks/TFG/NanoSam/TensorRT-10.0.1.6.Linux.x86_64-gnu.cuda-12.4.tar.gz" -C /usr/local/


TensorRT-10.0.1.6/
TensorRT-10.0.1.6/lib
TensorRT-10.0.1.6/bin
TensorRT-10.0.1.6/data/
TensorRT-10.0.1.6/data/int8_api/
TensorRT-10.0.1.6/data/int8_api/reference_labels.txt
TensorRT-10.0.1.6/data/int8_api/airliner.ppm
TensorRT-10.0.1.6/data/int8_api/README.md
TensorRT-10.0.1.6/data/int8_api/resnet50_per_tensor_dynamic_range.txt
TensorRT-10.0.1.6/data/char-rnn/
TensorRT-10.0.1.6/data/char-rnn/char-rnn.wts
TensorRT-10.0.1.6/data/char-rnn/model/
TensorRT-10.0.1.6/data/char-rnn/model/model-20080.meta
TensorRT-10.0.1.6/data/char-rnn/model/model-20080.data-00000-of-00001
TensorRT-10.0.1.6/data/char-rnn/model/model-20080.index
TensorRT-10.0.1.6/data/char-rnn/model/checkpoint
TensorRT-10.0.1.6/data/mnist/
TensorRT-10.0.1.6/data/mnist/9.pgm
TensorRT-10.0.1.6/data/mnist/4.pgm
TensorRT-10.0.1.6/data/mnist/6.pgm
TensorRT-10.0.1.6/data/mnist/mnist.onnx
TensorRT-10.0.1.6/data/mnist/3.pgm
TensorRT-10.0.1.6/data/mnist/5.pgm
TensorRT-10.0.1.6/data/mnist/2.pgm
TensorRT-10.0.1.6/data/mnist/README.md
Tens

In [10]:
!pip install numpy==1.24.4 --force-reinstall


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 114.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.24.4 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.24.4 which is incompatible.
pymc 5.23.0 requires numpy>=1.25.0, but you have numpy 1.24.4 which is incompatible.
blosc2 3.4.0 requires numpy>=1.26, but you have numpy 1.24.4 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.4 which is incompatible.
xarray-einstats 0.9.0 requires numpy>=1.25, but you have numpy 1.24.4 which is i

In [3]:
!python3 /content/nanosam/export_clean_onnx.py


/usr/local/lib/python3.11/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.11/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/content/nanosam/nanosam/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_5m_224 in registry with nanosam.mobile_sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/content/nanosam/nanosam/mobile_sam/modeling/tiny_vit_sam.py:656: UserWarning: Overwriting tiny_vit_11m_224 in registry with 

In [33]:
from nanosam.mobile_sam.build_sam import build_sam_vit_t

checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/TFG/NanoSam/mobile_sam.pt"
sam = build_sam_vit_t(checkpoint_path).to("cuda").eval()


In [40]:
# ✅ Forzar que todos los submódulos de MaskDecoder estén en cuda
mask_decoder = sam.mask_decoder
mask_decoder = mask_decoder.to("cuda")
mask_decoder.eval()


MaskDecoder(
  (transformer): TwoWayTransformer(
    (layers): ModuleList(
      (0-1): 2 x TwoWayAttentionBlock(
        (self_attn): Attention(
          (q_proj): Linear(in_features=256, out_features=256, bias=True)
          (k_proj): Linear(in_features=256, out_features=256, bias=True)
          (v_proj): Linear(in_features=256, out_features=256, bias=True)
          (out_proj): Linear(in_features=256, out_features=256, bias=True)
        )
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (cross_attn_token_to_image): Attention(
          (q_proj): Linear(in_features=256, out_features=128, bias=True)
          (k_proj): Linear(in_features=256, out_features=128, bias=True)
          (v_proj): Linear(in_features=256, out_features=128, bias=True)
          (out_proj): Linear(in_features=128, out_features=256, bias=True)
        )
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_fea

In [43]:
import torch
import torch.nn as nn

# ✅ Cargar MaskDecoder y mover a GPU correctamente
mask_decoder = sam.mask_decoder
mask_decoder.eval()
mask_decoder = mask_decoder.to("cuda")  # MUY IMPORTANTE HACERLO ANTES DE CREAR EL WRAPPER

# ✅ Crear wrapper en CUDA
mask_decoder_export = MaskDecoderONNXWrapper(mask_decoder).to("cuda")
mask_decoder_export.eval()

# ✅ Dummy inputs en CUDA
image_embeddings = torch.randn(1, 256, 64, 64, device="cuda", dtype=torch.float32)
image_pe = torch.randn(1, 256, 64, 64, device="cuda", dtype=torch.float32)
sparse_prompt_embeddings = torch.randn(1, 2, 256, device="cuda", dtype=torch.float32)
dense_prompt_embeddings = torch.randn(1, 256, 64, 64, device="cuda", dtype=torch.float32)
multimask_output_flag = torch.tensor([1.0], device="cuda", dtype=torch.float32)

# ✅ Ruta exportación
output_onnx_path = "/content/drive/MyDrive/Colab Notebooks/TFG/NanoSam/mobile_sam_mask_decoder_clean.onnx"

# ✅ Exportar ONNX
torch.onnx.export(
    mask_decoder_export,
    (
        image_embeddings,
        image_pe,
        sparse_prompt_embeddings,
        dense_prompt_embeddings,
        multimask_output_flag,
    ),
    output_onnx_path,
    input_names=[
        "image_embeddings", "image_pe", "sparse_prompt_embeddings", "dense_prompt_embeddings", "multimask_output_flag"
    ],
    output_names=["masks", "iou_predictions"],
    opset_version=14,
    do_constant_folding=True
)

print("✅ Exportación OK:", output_onnx_path)







/tmp/ipython-input-41-3483267291.py:27: TracerWarning: Converting a tensor to a Python number might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  multimask_output = bool(multimask_output_flag.item())


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [13]:
from nanosam.mobile_sam.build_sam import build_mobile_sam
import torch

# Ruta del checkpoint
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/TFG/NanoSam/mobile_sam.pt"

# 1️⃣ Crear el modelo vacío
model = build_mobile_sam(model_type="vit_t")
model.to("cuda")

# 2️⃣ Cargar pesos
state_dict = torch.load(checkpoint_path, map_location="cuda")
model.load_state_dict(state_dict)

# 3️⃣ Extraer MaskDecoder
mask_decoder = model.mask_decoder
mask_decoder.eval()
mask_decoder.to("cuda")

# Inputs dummy para la exportación
image_embeddings = torch.randn(1, 256, 64, 64, device="cuda", dtype=torch.float32)
point_coords = torch.randn(1, 1, 2, device="cuda", dtype=torch.float32)
point_labels = torch.randint(0, 2, (1, 1), device="cuda", dtype=torch.float32)
mask_input = torch.randn(1, 1, 256, 256, device="cuda", dtype=torch.float32)
has_mask_input = torch.tensor([1.0], device="cuda", dtype=torch.float32)
orig_im_size = torch.tensor([[512, 512]], device="cuda", dtype=torch.float32)

# Ruta de exportación
output_onnx_path = "/content/drive/MyDrive/Colab Notebooks/TFG/NanoSam/mobile_sam_clean.onnx"

# Exportación ONNX
torch.onnx.export(
    mask_decoder,
    (
        image_embeddings,
        point_coords,
        point_labels,
        mask_input,
        has_mask_input,
        orig_im_size
    ),
    output_onnx_path,
    input_names=[
        'image_embeddings', 'point_coords', 'point_labels',
        'mask_input', 'has_mask_input', 'orig_im_size'
    ],
    output_names=['masks', 'iou_predictions', 'low_res_masks'],
    opset_version=14,
    do_constant_folding=True,
    dynamic_axes={
        'point_coords': {1: 'num_points'},
        'point_labels': {1: 'num_points'}
    }
)

print(f"✅ Exportación terminada: {output_onnx_path}")




ImportError: cannot import name 'build_mobile_sam' from 'nanosam.mobile_sam.build_sam' (/content/nanosam/nanosam/mobile_sam/build_sam.py)

In [ ]:
!python3 /content/nanosam/export_clean_onnx.py \
  --checkpoint /content/drive/MyDrive/Colab\ Notebooks/TFG/NanoSam/mobile_sam_mask_decoder.pth \
  --output /content/drive/MyDrive/Colab\ Notebooks/TFG/NanoSam/mobile_sam_mask_decoder.onnx \
  --return-single-mask \
  --gelu-approximate \
  --use-stability-score \
  --opset 16


In [ ]:
!cat /content/nanosam/nanosam/mobile_sam/modeling/mask_decoder.py | grep class


In [ ]:
!export LD_LIBRARY_PATH=/usr/local/TensorRT-10.0.1.6/lib:$LD_LIBRARY_PATH && \
/usr/local/TensorRT-10.0.1.6/bin/trtexec \
    --onnx="/content/drive/MyDrive/Colab Notebooks/TFG/NanoSam/mobile_sam_mask_decoder.onnx" \
    --saveEngine="/content/drive/MyDrive/Colab Notebooks/TFG/NanoSam/mobile_sam_mask_decoder.engine" \
    --fp16 \
    --memPoolSize=workspace:4096
